In [5]:
# CELL 0: Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from scipy import stats
from matplotlib.patches import Wedge

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

plt.style.use('default')
sns.set_palette("husl")

import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="The default of observed=False is deprecated"
)


print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [6]:
# CELL 1: File Paths
base_path = r'/kaggle/input/nfl-dataset/114239_nfl_competition_files_published_analytics_final'
supp_path = os.path.join(base_path, "supplementary_data.csv")
train_folder = os.path.join(base_path, "train")
print(f"✅ Base path set: {base_path}")
print(f"✅ Supplementary data: {supp_path}")
print(f"✅ Training folder: {train_folder}")

✅ Base path set: /kaggle/input/nfl-dataset/114239_nfl_competition_files_published_analytics_final
✅ Supplementary data: /kaggle/input/nfl-dataset/114239_nfl_competition_files_published_analytics_final\supplementary_data.csv
✅ Training folder: /kaggle/input/nfl-dataset/114239_nfl_competition_files_published_analytics_final\train


In [7]:
# CELL 2: Load Data
print("⏳ Loading supplementary data...")
supp_df = pd.read_csv(supp_path)
print(f"✅ Loaded supplementary data: {supp_df.shape}")

print("\n⏳ Loading input tracking data (18 weeks)...")
input_files = [os.path.join(train_folder, f"input_2023_w{str(i).zfill(2)}.csv") for i in range(1, 19)]
input_df = pd.concat([pd.read_csv(f) for f in input_files], ignore_index=True)
print(f"✅ Loaded input data: {input_df.shape}")

print("\n⏳ Loading output tracking data (18 weeks)...")
output_files = [os.path.join(train_folder, f"output_2023_w{str(i).zfill(2)}.csv") for i in range(1, 19)]
output_df = pd.concat([pd.read_csv(f) for f in output_files], ignore_index=True)
print(f"✅ Loaded output data: {output_df.shape}")
print("\n✅ ALL DATA LOADED SUCCESSFULLY!")

⏳ Loading supplementary data...


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/nfl-dataset/114239_nfl_competition_files_published_analytics_final\\supplementary_data.csv'

In [ ]:
# CELL 3: Merge and Prepare Red Zone Data
print("⏳ Merging tracking data with play context...")
merged_df = pd.merge(input_df, supp_df, on=['game_id', 'play_id'], how='inner')
print(f"✅ Merged data: {merged_df.shape}")

def get_redzone_interval(yardline):
    if 5 <= yardline <= 10:
        return '5-10'
    elif 10 < yardline <= 15:
        return '10-15'
    elif 15 < yardline <= 20:
        return '15-20'
    else:
        return None

print("\n⏳ Filtering for red zone plays...")
merged_df['redzone_interval'] = merged_df['yardline_number'].apply(get_redzone_interval)
redzone_df = merged_df[merged_df['redzone_interval'].notna()].copy()
print(f"✅ Red zone plays: {redzone_df['play_id'].nunique()} unique plays")

print("\n✅ DATA PREPROCESSING COMPLETE!")

⏳ Merging tracking data with play context...
✅ Merged data: (4880579, 62)

⏳ Filtering for red zone plays...
✅ Red zone plays: 1982 unique plays

✅ DATA PREPROCESSING COMPLETE!


In [ ]:
# CELL 4: Play Summary
print("⏳ Creating play-level summary...")
play_summary = redzone_df.groupby(['game_id', 'play_id']).agg({
    'redzone_interval': 'first',
    'possession_team': 'first',
    'play_description': 'first',
    'offense_formation': 'first',
    'receiver_alignment': 'first',
    'route_of_targeted_receiver': 'first',
    'pass_result': 'first',
    'play_action': 'first',
    'dropback_type': 'first',
    'team_coverage_man_zone': 'first',
    'team_coverage_type': 'first',
    'yards_gained': 'first',
    'down': 'first',
    'yards_to_go': 'first'
}).reset_index()

print("⏳ Identifying touchdown plays...")
play_summary['is_touchdown'] = play_summary['play_description'].str.contains('TOUCHDOWN', case=False, na=False)
successful_plays = play_summary[play_summary['is_touchdown'] == True].copy()

print(f"✅ Total plays analyzed: {len(play_summary)}")
print(f"✅ Touchdown plays: {len(successful_plays)}")
print(f"✅ Overall TD rate: {len(successful_plays)/len(play_summary)*100:.1f}%")

print("\n✅ PLAY SUMMARY COMPLETE!")

⏳ Creating play-level summary...
⏳ Identifying touchdown plays...
✅ Total plays analyzed: 2692
✅ Touchdown plays: 318
✅ Overall TD rate: 11.8%

✅ PLAY SUMMARY COMPLETE!


In [ ]:
# CELL 5: Acceleration Helper
def calculate_accel_effort_pct(accel_yd_per_sec2):
    import math
    REDZONE_ACCEL_MAX = 2.5
    if accel_yd_per_sec2 is None:
        return None
    try:
        accel_val = float(accel_yd_per_sec2)
    except Exception:
        return None
    raw_ratio = accel_val / REDZONE_ACCEL_MAX if REDZONE_ACCEL_MAX != 0 else 0.0
    mapped = math.atan(raw_ratio) / (math.pi / 2)
    MIN_PCT = 75.0
    MAX_PCT = 100.0
    scaled_pct = MIN_PCT + mapped * (MAX_PCT - MIN_PCT)
    return round(scaled_pct, 1)

print("✅ Acceleration effort helper loaded.")

✅ Acceleration effort helper loaded.


In [ ]:
# CELL 6: Kinematics
FEET_TO_YARDS = 3.0

def calculate_receiver_kinematics_with_effort(tracking_data):
    if tracking_data.empty:
        return {
            'avg_accel': None,
            'avg_accel_effort_pct': None,
            'start_x': None,
            'start_y': None,
        }

    tracking_data = tracking_data.sort_values('frame_id')
    x = tracking_data['x'].values / FEET_TO_YARDS
    y = tracking_data['y'].values / FEET_TO_YARDS

    frame_interval = 0.01

    dx = np.diff(x)
    dy = np.diff(y)
    distance_per_frame = np.sqrt(dx**2 + dy**2) if len(dx) > 0 else np.array([])

    speed_per_frame = distance_per_frame / frame_interval if len(distance_per_frame) > 0 else np.array([])
    dv = np.diff(speed_per_frame) if len(speed_per_frame) > 1 else np.array([])
    acceleration_per_frame = dv / frame_interval if len(dv) > 0 else np.array([])

    avg_accel = np.mean(np.abs(acceleration_per_frame)) if len(acceleration_per_frame) > 0 else None
    avg_accel_effort_pct = calculate_accel_effort_pct(avg_accel) if avg_accel else None

    start_x = x[0] if len(x) > 0 else None
    start_y = y[0] if len(y) > 0 else None

    return {
        'avg_accel': round(avg_accel, 2) if avg_accel else None,
        'avg_accel_effort_pct': avg_accel_effort_pct,
        'start_x': start_x,
        'start_y': start_y,
    }

print("✅ Kinematics calculator loaded.")

✅ Kinematics calculator loaded.


In [ ]:
# CELL 7: Simulation Engine
def simulate_play_reliability(successes, attempts, global_avg_rate=None, simulations=10000):
    if global_avg_rate is None:
        if len(play_summary) > 0:
            global_avg_rate = max(len(successful_plays) / len(play_summary), 0.15)
        else:
            global_avg_rate = 0.15
    
    prior_strength = 8
    prior_alpha = max(global_avg_rate * prior_strength, 1)
    prior_beta = max((1 - global_avg_rate) * prior_strength, 1)
    
    posterior_alpha = prior_alpha + successes
    posterior_beta = prior_beta + (attempts - successes)
    
    simulated_rates = stats.beta.rvs(posterior_alpha, posterior_beta, size=simulations)
    
    expected_rate = np.mean(simulated_rates) * 100
    lower_bound_25 = np.percentile(simulated_rates, 25) * 100
    
    return round(expected_rate, 1), round(lower_bound_25, 1)

print("✅ Simulation engine loaded!")

✅ Simulation engine loaded!


In [ ]:
# Route mapping
ROUTE_ACCELERATION_MAP = {
    'post': 80,
    'go': 100,
    'cross': 70,
    'corner': 80,
    'wheel': 80,
    'angle': 60,
    'flat': 90,
}

def get_route_acceleration_pct(route_name):
    if pd.isna(route_name):
        return None
    route_lower = str(route_name).lower().strip()
    return ROUTE_ACCELERATION_MAP.get(route_lower, 75)

print("✅ Route-to-acceleration mapping loaded")

✅ Route-to-acceleration mapping loaded


In [ ]:
# CELL 8: Coverage Mapping
def map_coverage_input(user_input, available_covs):
    user_input = user_input.strip().upper()
    mapping = {
        'MAN': 'MAN_COVERAGE',
        'ZONE': 'ZONE_COVERAGE',
        'MAN_COVERAGE': 'MAN_COVERAGE',
        'ZONE_COVERAGE': 'ZONE_COVERAGE',
    }
    mapped = mapping.get(user_input, user_input)
    if mapped in available_covs:
        return mapped
    for cov in available_covs:
        if user_input in cov.upper():
            return cov
    return available_covs[0] if len(available_covs) > 0 else user_input

print("✅ Coverage mapper loaded!")

✅ Coverage mapper loaded!


In [ ]:
# CELL 9: Recommendation Engine
def get_enhanced_recommendations_final(yards_out, defense_type):
    if 5 <= yards_out <= 10:
        interval = '5-10'
    elif 10 < yards_out <= 15:
        interval = '10-15'
    elif 15 < yards_out <= 20:
        interval = '15-20'
    else:
        return {'error': 'Yards must be between 5-20'}

    available_covs = play_summary['team_coverage_man_zone'].unique()
    mapped_coverage = map_coverage_input(defense_type, available_covs)
    print(f"📊 Analyzing: {interval} yards vs {mapped_coverage}...")

    scenario_plays = successful_plays[
        (successful_plays['redzone_interval'] == interval) &
        (successful_plays['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    all_attempts_scenario = play_summary[
        (play_summary['redzone_interval'] == interval) &
        (play_summary['team_coverage_man_zone'] == mapped_coverage)
    ]
    
    if len(all_attempts_scenario) < 5:
        return {'error': f"Insufficient sample size ({len(all_attempts_scenario)} plays)."}

    grouped = all_attempts_scenario.groupby([
        'offense_formation', 
        'route_of_targeted_receiver',
        'receiver_alignment'
    ]).agg(
        total_attempts=('play_id', 'count'),
        td_count=('is_touchdown', 'sum')
    ).reset_index()

    grouped = grouped[grouped['total_attempts'] >= 1].copy()

    if grouped.empty:
        return {'error': "No patterns found."}

    results = []
    scenario_avg = len(scenario_plays) / max(len(all_attempts_scenario), 1)

    for _, row in grouped.iterrows():
        expected_rate, reliability_score = simulate_play_reliability(
            row['td_count'], 
            row['total_attempts'], 
            global_avg_rate=scenario_avg
        )
        
        raw_rate = (row['td_count'] / row['total_attempts']) * 100
        
        td_plays = scenario_plays[
            (scenario_plays['offense_formation'] == row['offense_formation']) &
            (scenario_plays['route_of_targeted_receiver'] == row['route_of_targeted_receiver']) &
            (scenario_plays['receiver_alignment'] == row['receiver_alignment'])
        ]
        
        play_ids = td_plays['play_id'].values
        pattern_tracking = redzone_df[redzone_df['play_id'].isin(play_ids)]
        
        receiver_stats_per_play = []
        
        for play_id in play_ids:
            play_tracking = pattern_tracking[pattern_tracking['play_id'] == play_id]
            for player in play_tracking['player_to_predict'].unique():
                player_tracking = play_tracking[play_tracking['player_to_predict'] == player]
                if len(player_tracking) > 1:
                    kinematics = calculate_receiver_kinematics_with_effort(player_tracking)
                    receiver_stats_per_play.append(kinematics)
        
        route_accel_pct = get_route_acceleration_pct(row['route_of_targeted_receiver'])
        
        if receiver_stats_per_play:
            start_x_vals = [r['start_x'] for r in receiver_stats_per_play if r['start_x'] is not None]
            start_y_vals = [r['start_y'] for r in receiver_stats_per_play if r['start_y'] is not None]
            
            start_x = np.mean(start_x_vals) if start_x_vals else None
            start_y = np.mean(start_y_vals) if start_y_vals else None
            pos_flex_x = np.std(start_x_vals) if len(start_x_vals) > 1 else None
            pos_flex_y = np.std(start_y_vals) if len(start_y_vals) > 1 else None
        else:
            start_x = start_y = pos_flex_x = pos_flex_y = None

        results.append({
            'formation': row['offense_formation'],
            'route': row['route_of_targeted_receiver'],
            'alignment': row['receiver_alignment'],
            'td_count': row['td_count'],
            'attempts': row['total_attempts'],
            'raw_success_rate': round(raw_rate, 1),
            'simulated_success': expected_rate,
            'reliability_score': reliability_score,
            'avg_accel_effort_pct': route_accel_pct,
            'start_x': round(start_x, 1) if start_x else None,
            'start_y': round(start_y, 1) if start_y else None,
            'pos_flex_x': round(pos_flex_x, 2) if pos_flex_x else None,
            'pos_flex_y': round(pos_flex_y, 2) if pos_flex_y else None,
        })

    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values('reliability_score', ascending=False)

    return {
        'scenario': {'interval': interval, 'defense': mapped_coverage},
        'data': df_results.head(5).to_dict('records')
    }

print("✅ Recommendation engine loaded!")

✅ Recommendation engine loaded!


In [ ]:
# CELL 11: Kaggle-Friendly Dashboard (Slider-Controlled Angles)

%matplotlib inline


# ===== INTERACTIVE WIDGETS FOR YARDS & DEFENSE (still fine on Kaggle) =====
print("\n")

yards_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=20,
    step=1,
    description='Yards Out:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

defense_dropdown = widgets.Dropdown(
    options=['ZONE_COVERAGE', 'MAN_COVERAGE'],
    value='ZONE_COVERAGE',
    description='Defense:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

analyze_button = widgets.Button(
    description='Analyze Play',
    button_style='success',
    tooltip='Click to analyze plays',
    icon='search',
    layout=widgets.Layout(width='300px', height='40px')
)

output_area = widgets.Output()

def on_analyze_click(b):
    with output_area:
        output_area.clear_output()
        result = get_enhanced_recommendations_final(yards_slider.value, defense_dropdown.value)
        display_final_results_enhanced(result)

analyze_button.on_click(on_analyze_click)

controls_box = widgets.VBox([
    widgets.HTML("<h3 style='text-align: center; color: #1a2a6c'>🏈 Red Zone Play Analyzer</h3>"),
    yards_slider,
    defense_dropdown,
    analyze_button,
    widgets.HTML("<hr>")
])

display(controls_box)
display(output_area)

# ===== ENHANCED DISPLAY FUNCTION =====
def display_final_results_enhanced(result):
    """Display recommendations with percentages, reliability, and receiver stats"""
    if 'error' in result:
        display(HTML(f"<h3 style='color: red'>{result['error']}</h3>"))
        return
    
    scenario = result['scenario']
    recs = result['data']
    
    if not recs or len(recs) == 0:
        display(HTML("<h3 style='color: red'>No plays found.</h3>"))
        return
    
    html_content = f"""
    <div style="background: linear-gradient(to right, #1a2a6c, #b21f1f, #fdbb2d); padding: 20px; border-radius: 10px; color: white;">
        <h2 style="margin: 0">🏈 TOP RANKED PLAYS {scenario['interval']} YDS vs {scenario['defense']}</h2>
    </div>
    """
    
    for i, rec in enumerate(recs):
        rank = i + 1
        rel_score = rec.get('reliability_score', 0)
        rel_color = '#2ecc71' if rel_score >= 30 else '#f1c40f'
        
        success_pct = rec.get('simulated_success', 0)
        reliability_pct = rec.get('reliability_score', 0)
        raw_pct = rec.get('raw_success_rate', 0)
        accel_pct = rec.get('avg_accel_effort_pct', 0)
        
        start_x = rec.get('start_x', 'N/A')
        start_y = rec.get('start_y', 'N/A')
        pos_flex_x = rec.get('pos_flex_x', 'N/A')
        pos_flex_y = rec.get('pos_flex_y', 'N/A')
        
        html_content += f"""
        <div style="border: 1px solid ddd; margin-top: 20px; border-radius: 8px; overflow: hidden; box-shadow: 0 4px 6px rgba(0,0,0,0.1)">
            <div style="background-color: f8f9fa; padding: 15px; border-left: 6px solid {rel_color}">
                <h3 style="margin: 0; color: #333">{rank}. {rec.get('formation', 'Unknown')} | {rec.get('route', 'Unknown').upper()} | {rec.get('alignment', 'Unknown')}</h3>
            </div>
            <div style="padding: 15px; background-color: white">
                <div style="display: grid; grid-template-columns: 1fr 1fr 1fr 1fr; gap: 15px; margin-bottom: 20px">
                    <div style="text-align: center; border-right: 1px solid eee">
                        <div style="font-weight: bold; font-size: 22px; color: {rel_color}">{success_pct:.0f}%</div>
                        <div style="font-size: 11px; color: #888">Expected Success Rate</div>
                    </div>
                    <div style="text-align: center; border-right: 1px solid eee">
                        <div style="font-weight: bold; font-size: 22px; color: {rel_color}">{reliability_pct:.0f}%</div>
                        <div style="font-size: 11px; color: #888">Reliability Score</div>
                    </div>
                    <div style="text-align: center; border-right: 1px solid eee">
                        <div style="font-weight: bold; font-size: 18px">{raw_pct:.0f}%</div>
                        <div style="font-size: 11px; color: #888">Raw Stats ({rec.get('td_count', 0)}/{rec.get('attempts', 0)})</div>
                    </div>
                    <div style="text-align: center">
                        <div style="font-weight: bold; font-size: 22px; color: #3498db">{accel_pct:.0f}%</div>
                        <div style="font-size: 11px; color: #888">Receiver Acceleration</div>
                    </div>
                </div>
                <div style="border-top: 1px solid #eee; padding-top: 15px">
                    <div style="font-weight: bold; margin-bottom: 10px; color: #1a2a6c">📍 Receiver Position & Movement</div>
                    <div style="display: grid; grid-template-columns: 1fr 1fr 1fr 1fr; gap: 15px; font-size: 12px">
                        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; text-align: center">
                            <div style="font-weight: bold; color: #333">{start_x}</div>
                            <div style="color: #888">Start X (yards)</div>
                        </div>
                        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; text-align: center">
                            <div style="font-weight: bold; color: #333">{start_y}</div>
                            <div style="color: #888">Start Y (yards)</div>
                        </div>
                        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; text-align: center">
                            <div style="font-weight: bold; color: #333">{pos_flex_x}</div>
                            <div style="color: #888">X Position Variance</div>
                        </div>
                        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; text-align: center">
                            <div style="font-weight: bold; color: #333">{pos_flex_y}</div>
                            <div style="color: #888">Y Position Variance</div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
        """
    
    display(HTML(html_content))

print("⏳ Building Kaggle-friendly dashboard with angle sliders...")

try:
    import ipywidgets as widgets
    from ipywidgets import interactive_output
    import matplotlib.gridspec as gridspec

    # ===== PART 1: DEFENDER DATA (same as before) =====
    last_positions = redzone_df.groupby(['game_id', 'play_id', 'player_to_predict']).agg({
        'x': 'last',
        'y': 'last'
    }).reset_index()
    
    receiver_last = last_positions[last_positions['player_to_predict'] == True].copy()
    receiver_last = receiver_last.rename(columns={'x': 'caught_x', 'y': 'caught_y'})
    receiver_last = receiver_last[['game_id', 'play_id', 'caught_x', 'caught_y']]
    
    defender_last = last_positions[last_positions['player_to_predict'] == False].copy()
    defender_last = defender_last.merge(receiver_last, on=['game_id', 'play_id'], how='inner')
    
    play_info = redzone_df[['game_id', 'play_id', 'play_direction', 'pass_result']].drop_duplicates()
    with_direction = defender_last.merge(play_info, on=['game_id', 'play_id'], how='left')
    
    with_direction['x_rel'] = with_direction['x'] - with_direction['caught_x']
    with_direction['y_rel'] = with_direction['y'] - with_direction['caught_y']
    
    def apply_degrees_dynamic(row, angle1=115, angle2=245):
        x = row['x_rel']
        y = row['y_rel']
        direction = row['play_direction']
        try:
            theta = np.degrees(np.arctan2(y, x))
            theta = np.mod(theta, 360)
            if direction == 'right':
                if angle1 <= theta <= angle2:
                    return 'front'
                else:
                    return 'behind'
            elif direction == 'left':
                if angle1 <= theta <= angle2:
                    return 'behind'
                else:
                    return 'front'
            else:
                return 'unknown'
        except:
            return 'error'
    
    with_direction['defensive_distance'] = np.sqrt(with_direction['x_rel']**2 + with_direction['y_rel']**2)
    with_direction['dist_bin'] = pd.cut(with_direction['defensive_distance'], bins=10)

    # ===== HELPERS THAT USE angle1, angle2 =====
    def get_front_behind_dfs(a1, a2):
        with_direction['d_pos_dynamic'] = with_direction.apply(
            lambda row: apply_degrees_dynamic(row, a1, a2), axis=1
        )
        front_df = with_direction[with_direction['d_pos_dynamic'] == 'front'].copy()
        behind_df = with_direction[with_direction['d_pos_dynamic'] == 'behind'].copy()
        return front_df, behind_df

    def draw_dashboard(angle1=115, angle2=245):
        """Draw the full dashboard for given angles (Kaggle-safe)."""
        plt.close('all')
        fig = plt.figure(figsize=(14, 9))
        gs = gridspec.GridSpec(3, 2, figure=fig,
                               height_ratios=[1, 1, 1.2],
                               hspace=0.35, wspace=0.25)

        # --- Polar subplot (left, top 2 rows) ---
        ax_polar = fig.add_subplot(gs[0:2, 0], projection='polar')
        ax_polar.set_ylim(0, 1.3)
        ax_polar.set_theta_zero_location('E')
        ax_polar.set_theta_direction(1)
        ax_polar.set_title('RED/BLUE Lines\n(Front Defender Range)', fontsize=10,
                           fontweight='bold', pad=10)
        ax_polar.grid(True, alpha=0.2)

        theta_start = np.radians(angle1)
        theta_end = np.radians(angle2)
        theta_range = np.linspace(theta_start, theta_end, 40)
        radii = np.linspace(0, 1.2, 20)
        theta_grid, r_grid = np.meshgrid(theta_range, radii)
        z = np.ones_like(theta_grid)
        ax_polar.contourf(theta_grid, r_grid, z, levels=[0.5, 1.5],
                          colors=['#90EE90'], alpha=0.25)

        ax_polar.plot([theta_start, theta_start], [0, 1.2],
                      color='red', linewidth=5, label='Start (RED)', zorder=10)
        ax_polar.plot([theta_end, theta_end], [0, 1.2],
                      color='blue', linewidth=5, label='End (BLUE)', zorder=10)
        ax_polar.legend(loc='upper left', fontsize=8, framealpha=0.9)

        # --- Catch rate bars (right, top 2 rows) ---
        ax_compare = fig.add_subplot(gs[0:2, 1])
        front_df, behind_df = get_front_behind_dfs(angle1, angle2)

        front_rate = (front_df['pass_result'] == 'C').mean() if len(front_df) > 0 else 0
        behind_rate = (behind_df['pass_result'] == 'C').mean() if len(behind_df) > 0 else 0

        bars = ax_compare.bar(['In Front', 'Behind'], [front_rate, behind_rate],
                              color=['#e74c3c', '#2ecc71'],
                              width=0.4, edgecolor='black',
                              linewidth=1.5, alpha=0.85)
        ax_compare.set_ylabel('Catch Rate', fontsize=9, fontweight='bold')
        ax_compare.set_ylim([0, 1.0])
        ax_compare.set_title(f'Catch Rate\n({angle1:.0f}° to {angle2:.0f}°)',
                             fontsize=10, fontweight='bold')
        ax_compare.grid(True, alpha=0.2, axis='y')
        ax_compare.tick_params(labelsize=8)
        for bar, rate in zip(bars, [front_rate, behind_rate]):
            h = bar.get_height()
            ax_compare.text(bar.get_x() + bar.get_width()/2., h + 0.02,
                            f'{rate:.1%}', ha='center', va='bottom',
                            fontsize=9, fontweight='bold')

        # --- Distance breakdown (bottom full width) ---
        ax_dist = fig.add_subplot(gs[2, :])
        prop_f = front_df.groupby('dist_bin')['pass_result'].apply(
            lambda x: (x == 'C').mean() if len(x) > 0 else 0
        ).reset_index(name='catch_rate')
        prop_b = behind_df.groupby('dist_bin')['pass_result'].apply(
            lambda x: (x == 'C').mean() if len(x) > 0 else 0
        ).reset_index(name='catch_rate')

        if len(prop_f) > 0:
            prop_f['bin_center'] = prop_f['dist_bin'].apply(
                lambda x: x.mid if pd.notna(x.mid) else 0
            )
            ax_dist.plot(prop_f['bin_center'], prop_f['catch_rate'],
                         marker='s', label='In Front', linewidth=2,
                         markersize=5, color='#e74c3c', zorder=3)

        if len(prop_b) > 0:
            prop_b['bin_center'] = prop_b['dist_bin'].apply(
                lambda x: x.mid if pd.notna(x.mid) else 0
            )
            ax_dist.plot(prop_b['bin_center'], prop_b['catch_rate'],
                         marker='o', label='Behind', linewidth=2,
                         markersize=5, color='#2ecc71', zorder=3)

        ax_dist.set_xlabel('Distance (yards)', fontsize=9, fontweight='bold')
        ax_dist.set_ylabel('Catch Rate', fontsize=9, fontweight='bold')
        ax_dist.set_title('By Defensive Position', fontsize=10,
                          fontweight='bold')
        ax_dist.legend(fontsize=8, loc='best', framealpha=0.9)
        ax_dist.grid(True, alpha=0.2, linestyle='--')
        ax_dist.set_ylim([0, 1.0])
        ax_dist.tick_params(labelsize=8)
        ax_dist.margins(x=0.1)

        plt.show()

    # ===== ANGLE SLIDERS (instead of drag events) =====
    angle1_slider = widgets.IntSlider(
        value=115, min=0, max=359, step=1,
        description='Start angle', style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )
    angle2_slider = widgets.IntSlider(
        value=245, min=0, max=359, step=1,
        description='End angle', style={'description_width': '100px'},
        layout=widgets.Layout(width='400px')
    )

    ui = widgets.VBox([
        widgets.HTML("<h3>Front Defender Angle Range (Kaggle interactive via sliders)</h3>"),
        angle1_slider,
        angle2_slider
    ])

    out = interactive_output(
        draw_dashboard,
        {'angle1': angle1_slider, 'angle2': angle2_slider}
    )

    display(ui, out)

except Exception as e:
    print(f"⚠️  Error: {e}")
    import traceback
    traceback.print_exc()


Output()

⏳ Building Kaggle-friendly dashboard with angle sliders...
⚠️  Error: name 'redzone_df' is not defined


Traceback (most recent call last):
  File "C:\Users\dmath\AppData\Local\Temp\ipykernel_43324\68048761.py", line 150, in <module>
    last_positions = redzone_df.groupby(['game_id', 'play_id', 'player_to_predict']).agg({
                     ^^^^^^^^^^
NameError: name 'redzone_df' is not defined
